In [1]:
import numpy as np
import pandas as pd

from routines import ReadCOSMOthermGamma, ReadCOSMOthermExcess, ReadCOSMOthermIsotherm

param_str = 'TZVP'

fes_vec = ['c2mim_hdfs_pfpa', 'tba_nfs_pfpa', 'tbp_br_pfpa']
hfc_vec = ['difluoromethane', '1,1,1,2-tetrafluoroethane', 'pentafluoroethane']
hfc_nom = {'difluoromethane': 'R32', '1,1,1,2-tetrafluoroethane': 'R134a', 'pentafluoroethane': 'R125'}
rdes_str = '[1_2]'

T = 303.15

# Activity coefficients

In [2]:
inp_str = 'activity_coefficients/'
out_str = 'activity_coefficients/'

for i,fes in enumerate(fes_vec):
    COSMO = ReadCOSMOthermGamma(inp_str+fes+'-ActivityCoefficients-'+param_str+'.tab')
    for j,hfc in enumerate(hfc_vec):
        x1, gamma = np.array(COSMO[(T)][0]), np.array(COSMO[(T)][1])
        x1, gamma = np.array(x1)*2, np.array(gamma)
        x1, gamma = np.append(x1, 0), np.append(gamma, 1)
        sort_ndx = np.argsort(x1)
        x1, gamma = x1[sort_ndx], gamma[sort_ndx]
        np.savetxt(out_str+fes+'-'+param_str+'.out', np.array([x1, gamma]).T, header='x1\tg_HBD', fmt='%.8f', delimiter='\t')

# Excess properties

In [3]:
inp_str = 'excess_properties/'
out_str = 'excess_properties/'

x1, HE, GE, HE_mf, HE_hb, HE_vdw, HE_conf = [], [], [], [], [], [], []
for i,fes in enumerate(fes_vec):
    HE_aux, GE_aux, HE_mf_aux, HE_hb_aux, HE_vdw_aux, HE_conf_aux = [], [], [], [], [], []
    data_inp = inp_str+fes+rdes_str+'-ExcessProperties-'+param_str+'.tab'
    COSMO = ReadCOSMOthermExcess(data_inp)
    for j,hfc in enumerate(hfc_vec):
        x1.append(COSMO[(T, hfc)][0][0])
        HE_aux.append(COSMO[(T, hfc)][1][0]), GE_aux.append(COSMO[(T, hfc)][2][0])
        HE_mf_aux.append(COSMO[(T, hfc)][3][0]), HE_hb_aux.append(COSMO[(T, hfc)][4][0])
        HE_vdw_aux.append(COSMO[(T, hfc)][5][0]), HE_conf_aux.append(COSMO[(T, hfc)][6][0])
    HE.append(HE_aux), GE.append(GE_aux)
    HE_mf.append(HE_mf_aux), HE_hb.append(HE_hb_aux), HE_vdw.append(HE_vdw_aux), HE_conf.append(HE_conf_aux)

for i,fes in enumerate(fes_vec):
    with open('excess_properties/'+fes+rdes_str+'-'+param_str+'.out', 'w') as f:
        f.write('# HFC\tG^E/kJ/mol\tH^E/kJ/mol\tH^E_vdw/kJ/mol\tH^E_hb/kJ/mol\tH^E_mf/kJ/mol\n')
        for j,hfc in enumerate(hfc_vec):
            f.write(f'{hfc_nom[hfc]}\t{GE[i][j]:.8f}\t{HE[i][j]:.8f}\t{HE_vdw[i][j]:.8f}\t{HE_hb[i][j]:.8f}\t{HE_mf[i][j]:.8f}\n')

# Solubility isotherms

In [4]:
inp_str = 'isotherms/'
out_str = 'isotherms/'

for i,fes in enumerate(fes_vec):
    COSMO = ReadCOSMOthermIsotherm(inp_str+fes+rdes_str+'-SolubilityIsotherms-'+param_str+'.tab')
    for j,hfc in enumerate(hfc_vec):
        X1, Y1, P = np.array(COSMO[(T, hfc)][0]), np.array(COSMO[(T, hfc)][1]), np.array(COSMO[(T, hfc)][2])
        np.savetxt(out_str+hfc_nom[hfc]+'+'+fes+rdes_str+'-'+param_str+'.out', np.array([X1, Y1, P]).T, header='x1\ty1\tP/MPa', delimiter='\t', fmt='%.8f')

# $\sigma$-profiles

In [14]:
mol_vec = ['difluoromethane', '1,1,1,2-tetrafluoroethane', 'pentafluoroethane', 
                '1-ethyl-3-methyl-imidazolium', 'perfluorooctanesulfonate', 'tetra-n-butylammonium', 'perfluorobutanesulfonate', 'tetra-n-butylphosphonium', 'br', 
                'perfluoropentanoic_acid']
inp_str = 'sigma_profiles/'
out_str = 'sigma_profiles/'

data = pd.read_excel(inp_str+param_str+'.xlsx', sheet_name=None)
svec, psvec = [], []
for i in range(len(mol_vec)):
    svec = data[mol_vec[i]]['s [e/A^2]'].values
    psvec.append(data[mol_vec[i]]['p(s)'].values)
ps_r32, ps_r134a, ps_r125, ps_c2m, ps_hdfs, ps_tba, ps_nfs, ps_tbp, ps_br, ps_pfpa = psvec
msvec = []
for i,fes in enumerate(fes_vec):
    msvec.append(data[fes+'-1_2']['m(s)'].values)
ms_fes1, ms_fes2, ms_fes3 = msvec
xfes = np.array([1, 1, 2])/np.sum([1, 1, 2])
ps_fes1 = (xfes[0]*ps_c2m + xfes[1]*ps_hdfs + xfes[2]*ps_pfpa)
ps_fes2 = (xfes[0]*ps_tba +  xfes[1]*ps_nfs + xfes[2]*ps_pfpa)
ps_fes3 = (xfes[0]*ps_tbp +   xfes[1]*ps_br + xfes[2]*ps_pfpa)

ps_con = [ps_c2m, ps_tba, ps_tbp]
ps_aon = [ps_hdfs, ps_nfs, ps_br]
ps_hbd = ps_pfpa
np.savetxt(out_str+'sprof_fes-'+param_str+'.out', 
           np.array([svec, ps_con[0], ps_aon[0], ps_con[1], ps_aon[1], ps_con[2], ps_aon[2], ps_hbd, ps_fes1, ps_fes2, ps_fes3]).T,
              header='  s/e/A2\t    pC2mim\t     pHDFS\t      pTBA\t      pNFS\t      pTBP\t       pBr\t     pPFPA\t     pFES1\t     pFES2\t     pFES3',
              fmt='%10.5f', delimiter='\t')
np.savetxt(out_str+'spot_fes-'+param_str+'.out',
           np.array([svec, ms_fes1, ms_fes2, ms_fes3]).T,
              header='  s/e/A2\t    muFES1\t    muFES2\t    muFES3',
              fmt='%10.5f', delimiter='\t')